Get Power BI refresh logs and metadata

## Import Library

In [ ]:
# Import library
import requests
import json
import pandas as pd
import msal
from datetime import date
from notebookutils import mssparkutils

## Set default Parameters

In [ ]:
# Set default parameter 
client_id = ""
client_secret = ""
tenant_name = ""
authority_url = "https://login.microsoftonline.com/" + tenant_name 
scope = ['https://analysis.windows.net/powerbi/api/.default']

# When using Synapse Notebook to get values in Azure Key Value
client_id = mssparkutils.credentials.getSecret('key-vault-name','secret-name','key-vault-linked-entity-name')
client_secret = mssparkutils.credentials.getSecret('key-vault-name','secret-name','key-vault-linked-entity-name')
tenant_name = mssparkutils.credentials.getSecret('key-vault-name','secret-name','key-vault-linked-entity-name') 

## Get token using MASL and Notebookutil package

In [ ]:
#Use MSAL to grab token
app = msal.ConfidentialClientApplication(client_id, authority=authority_url, client_credential=client_secret)
result = app.acquire_token_for_client(scopes=scope)

#Get latest Power BI Dataset Refresh
access_token = result['access_token']

# Set header
header = {
    'Content-Type':'application/json', 
    'Authorization':f'Bearer {access_token}'
    }

## Get Token without additional packages

In [ ]:
import get_token

access_token = get_token()

# Query data

## Get all workspaces

In [ ]:
get_all_workspaces_url = 'https://api.powerbi.com/v1.0/myorg/groups'
api_call = requests.get(url=get_all_workspaces_url, headers=header)
df_get_all_workspaces = pd.DataFrame.from_dict(api_call.json()['value'], orient ='columns')

## Get all datasets from list of workspaces

In [ ]:
# Get all datasets

# Set workspace list
workspace_id_list = df_get_all_workspaces['id']
# Define an empty dataframe
df_get_all_datasets = pd.DataFrame()
# Loop through workspace
for workspace_id in workspace_id_list:
    workspace_name = df_get_all_workspaces.query('id == "{0}"'.format(workspace_id))["name"].iloc[0]
    # Define URL endpoint
    get_all_datasets_url =  'https://api.powerbi.com/v1.0/myorg/groups/{0}/datasets'.format(workspace_id)
    # Send API call to get data
    api_call = requests.get(url=get_all_datasets_url, headers=header)
    # If api_call success then proceed:
    if api_call.status_code == 200:
        # Create dataframe to store data
        df = pd.DataFrame.from_dict(api_call.json()['value'], orient ='columns')
        # Add workspace id column
        df['workspaceId'] = workspace_id
        # Add workspace name column
        df['workspaceName'] = workspace_name
        # Append data
        df_get_all_datasets = df_get_all_datasets.append(df)

## Get all dataflows from list of workspaces

In [ ]:
# Set workspace list
workspace_id_list = df_get_all_workspaces['id']
# Define an empty dataframe
df_get_all_dataflows = pd.DataFrame()
# Loop through workspace
for workspace_id in workspace_id_list:
    workspace_name = df_get_all_workspaces.query('id == "{0}"'.format(workspace_id))["name"].iloc[0]
    # Define URL endpoint
    get_all_dataflows_url =  'https://api.powerbi.com/v1.0/myorg/groups/{0}/dataflows'.format(workspace_id)
    # Send API call to get data
    api_call = requests.get(url=get_all_dataflows_url, headers=header)
    # If api_call success then proceed:
    if api_call.status_code == 200:
        # Create dataframe to store data
        df = pd.DataFrame.from_dict(api_call.json()['value'], orient ='columns')
        # Add column
        df['workspaceId'] = workspace_id
        # Add workspace name column
        df['workspaceName'] = workspace_name
        # Append data
        df_get_all_dataflows = df_get_all_dataflows.append(df)

## Get all reports from list of workspaces

In [ ]:
# Set workspace list
workspace_id_list = df_get_all_workspaces['id']
# Define an empty dataframe
df_get_all_reports = pd.DataFrame()
# Loop through workspace
for workspace_id in workspace_id_list:
    workspace_name = df_get_all_workspaces.query('id == "{0}"'.format(workspace_id))["name"].iloc[0]
    # Define URL endpoint
    get_all_reports_url =  'https://api.powerbi.com/v1.0/myorg/groups/{0}/reports'.format(workspace_id)
    # Send API call to get data
    api_call = requests.get(url=get_all_reports_url, headers=header)
    # If api_call success then proceed:
    if api_call.status_code == 200:
        # Create dataframe to store data
        df = pd.DataFrame.from_dict(api_call.json()['value'], orient ='columns')
        # Add workspace name column
        df['workspaceName'] = workspace_name
        # Append data
        df_get_all_reports = df_get_all_reports.append(df)

## Get all dataset refresh history from list of workspaces

In [ ]:
# Get dataset refresh history
df_get_all_datasets_refresh_history = pd.DataFrame()
list_of_ds = df_get_all_datasets.query('isRefreshable == True')['id']
# Loop through dataset
for dataset_id in list_of_ds:
    # Get workspace id
    workspace_id = df_get_all_datasets.query('id == "{0}"'.format(dataset_id))["workspaceId"].iloc[0]
    # Get workspace name
    workspace_name = df_get_all_datasets.query('id == "{0}"'.format(dataset_id))["workspaceName"].iloc[0]
    # Get dataset name
    dataset_name = df_get_all_datasets.query('id == "{0}"'.format(dataset_id))["name"].iloc[0]
    # Define URL endpoint
    get_all_dataset_refresh_history_url =  'https://api.powerbi.com/v1.0/myorg/groups/{0}/datasets/{1}/refreshes?$top=10'.format(workspace_id,dataset_id)
    # Send API to get data
    api_call = requests.get(url=get_all_dataset_refresh_history_url, headers=header)
    # If api_call success then proceed:
    if api_call.status_code == 200:
        # Parse data from json output
        df = pd.DataFrame.from_dict(api_call.json()['value'], orient ='columns')
        # Add column
        df['datasetId'] = dataset_id
        # Add column
        df['datasetName'] = dataset_name
        # Add column
        df['workspaceId'] = workspace_id
        # Add column
        df['workspaceName'] = workspace_name
        # Append data
        df_get_all_datasets_refresh_history = df_get_all_datasets_refresh_history.append(df)


## Get all dataflow refresh history from list of workspaces

In [ ]:
# Get dataflow refresh history
df_get_all_dataflows_refresh_history = pd.DataFrame()
list_of_dataflows = df_get_all_dataflows['objectId']

# Loop through each workspace
for workspace_id in workspace_id_list:
    # Loop through dataflow
    for dataflow_id in list_of_dataflows:
        # Get workspace name
        workspace_name = df_get_all_dataflows.query('objectId == "{0}"'.format(dataflow_id))["workspaceName"].iloc[0]
        # Get dataflow name
        dataflow_name = df_get_all_dataflows.query('objectId == "{0}"'.format(dataflow_id))["name"].iloc[0]
        # Define URL endpoint
        get_all_dataflow_refresh_history_url =  'https://api.powerbi.com/v1.0/myorg/groups/{0}/dataflows/{1}/transactions'.format(workspace_id,dataflow_id)
        # Send API to get data
        api_call = requests.get(url=get_all_dataflow_refresh_history_url, headers=header)
        # If api_call success then proceed:
        if api_call.status_code == 200:
            # Parse data from json output
            df = pd.DataFrame.from_dict(api_call.json()['value'], orient ='columns')
            # Add column
            df['dataflowId'] = dataflow_id
            # Add column
            df['dataflowName'] = dataflow_name
            # Add column
            df['workspaceId'] = workspace_id
            # Add column
            df['workspaceName'] = workspace_name
            # Append data
            df_get_all_dataflows_refresh_history = df_get_all_dataflows_refresh_history.append(df)

## Get all dataset refresh schedule from list of workspaces

In [ ]:
# Get dataset refresh schedule
df_get_all_datasets_refresh_schedule = pd.DataFrame()
list_of_ds = df_get_all_datasets.query('isRefreshable == True')['id']
# Loop through dataset
for dataset_id in list_of_ds:
    # Get workspace id
    workspace_id = df_get_all_datasets.query('id == "{0}"'.format(dataset_id))["workspaceId"].iloc[0]
    # Get workspace name
    workspace_name = df_get_all_datasets.query('id == "{0}"'.format(dataset_id))["workspaceName"].iloc[0]
    # Get dataset name
    dataset_name = df_get_all_datasets.query('id == "{0}"'.format(dataset_id))["name"].iloc[0]
    # Define URL endpoint
    get_all_dataset_refresh_schedule_url = 'https://api.powerbi.com/v1.0/myorg/groups/{0}/datasets/{1}/refreshSchedule'.format(workspace_id,dataset_id)
    api_call = requests.get(url=get_all_dataset_refresh_schedule_url, headers=header)
    # If api_call success then proceed:
    if api_call.status_code == 200:
    # Create dictionary to store json data
        days_json = '|'.join(api_call.json()['days'])
        times_json = '|'.join(api_call.json()['times'])
        enabled_json = api_call.json()['enabled']
        localTimeZoneId_json = api_call.json()['localTimeZoneId']

        this_dict = {
            "days": [days_json],
            "time": [times_json],
            "enabled": [enabled_json],
            "localTimeZoneId": [localTimeZoneId_json]
        }
        # Convert dict to data frame
        df = pd.DataFrame.from_dict(this_dict, orient ='columns')
        # Add column
        df['datasetId'] = dataset_id
        # Add column
        df['datasetName'] = dataset_name
        # Add column
        df['workspaceId'] = workspace_id
        # Add column
        df['workspaceName'] = workspace_name
        # Append data
        df_get_all_datasets_refresh_schedule = df_get_all_datasets_refresh_schedule.append(df)

# Export data to CSV

## Write all workspaces to CSV

In [ ]:
df_get_all_workspaces.to_csv(r"workspaces/workspaces.csv",index=False)

## Write all dataflows to CSV

In [ ]:
df_get_all_dataflows.to_csv(r"dataflows/dataflows.csv",index=False)

## Write all datasets to CSV

In [ ]:
df_get_all_datasets.to_csv(r"powerbi/datasets/datasets.csv",index=False)

## Write all reports info to CSV

In [ ]:
df_get_all_reports.to_csv(r"powerbi/reports/reports.csv",index=False)

## Write dataset refresh schedule info to CSV

In [ ]:
df_get_all_datasets_refresh_schedule.to_csv(r"powerbi/datasets_refresh_schedule/refresh_schedule.csv",index=False)

## Write dataset refresh history info to CSV

In [ ]:
df_get_all_datasets_refresh_history.to_csv(r"powerbi/datasets_refresh_history/refresh_history_snapshot@{0}.csv".format(date.today()),index=False)

## Write dataflow refresh history info to CSV

In [ ]:
df_get_all_dataflows_refresh_history.to_csv(r"powerbi/dataflows_refresh_history/refresh_history_snapshot@{0}.csv".format(date.today()),index=False)